# Project 3 Team 5
## Date: 17 January 2025 

1. Project Overview
2. Data Load and preprocessing
3. Model training
4. Evaluation
5. Deployment

### 1. Project Overview

### 2. Data load and preprocessing 
#### Packages

In [1]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lebog\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lebog\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lebog\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Datasets

In [26]:
#importing the dataset:the train dataset 
train_df = pd.read_csv("Data/processed/train.csv")
train_df.head(5)

,headlines,description,content,url,category
0,RBI revises definition of politically-exposed ...,The central bank has also asked chairpersons a...,The Reserve Bank of India (RBI) has changed th...,https://indianexpress.com/article/business/ban...,business
1,NDTV Q2 net profit falls 57.4% to Rs 5.55 cror...,NDTV's consolidated revenue from operations wa...,Broadcaster New Delhi Television Ltd on Monday...,https://indianexpress.com/article/business/com...,business
2,"Akasa Air ‘well capitalised’, can grow much fa...",The initial share sale will be open for public...,Homegrown server maker Netweb Technologies Ind...,https://indianexpress.com/article/business/mar...,business
3,India’s current account deficit declines sharp...,The current account deficit (CAD) was 3.8 per ...,India’s current account deficit declined sharp...,https://indianexpress.com/article/business/eco...,business
4,"States borrowing cost soars to 7.68%, highest ...",The prices shot up reflecting the overall high...,States have been forced to pay through their n...,https://indianexpress.com/article/business/eco...,business


In [3]:
#importing the dataset: the test 
test_df = pd.read_csv("Data/processed/test.csv")
test_df.head(5)

,headlines,description,content,url,category
0,NLC India wins contract for power supply to Ra...,State-owned firm NLC India Ltd (NLCIL) on Mond...,State-owned firm NLC India Ltd (NLCIL) on Mond...,https://indianexpress.com/article/business/com...,business
1,SBI Clerk prelims exams dates announced; admit...,SBI Clerk Prelims Exam: The SBI Clerk prelims ...,SBI Clerk Prelims Exam: The State Bank of Indi...,https://indianexpress.com/article/education/sb...,education
2,"Golden Globes: Michelle Yeoh, Will Ferrell, An...","Barbie is the top nominee this year, followed ...","Michelle Yeoh, Will Ferrell, Angela Bassett an...",https://indianexpress.com/article/entertainmen...,entertainment
3,"OnePlus Nord 3 at Rs 27,999 as part of new pri...",New deal makes the OnePlus Nord 3 an easy purc...,"In our review of the OnePlus Nord 3 5G, we pra...",https://indianexpress.com/article/technology/t...,technology
4,Adani family’s partners used ‘opaque’ funds to...,Citing review of files from multiple tax haven...,Millions of dollars were invested in some publ...,https://indianexpress.com/article/business/ada...,business


#### 2.1. Data inspection

In [4]:
print("The dimensions for the Train set are:", train_df.shape)
print("The dimensions for the test set are:",test_df.shape)

The dimensions for the Train set are: (5520, 5)
The dimensions for the test set are: (2000, 5)


In [5]:
display(train_df.info(), test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5520 entries, 0 to 5519
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   headlines    5520 non-null   object
 1   description  5520 non-null   object
 2   content      5520 non-null   object
 3   url          5520 non-null   object
 4   category     5520 non-null   object
dtypes: object(5)
memory usage: 215.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   headlines    2000 non-null   object
 1   description  2000 non-null   object
 2   content      2000 non-null   object
 3   url          2000 non-null   object
 4   category     2000 non-null   object
dtypes: object(5)
memory usage: 78.3+ KB


None

None

There are no missing values in both the train and test dataset

In [6]:
null_counts = pd.DataFrame({
    "Train Null Count": train_df.isnull().sum(),
    "Test Null Count": test_df.isnull().sum()
})
print(null_counts)

             Train Null Count  Test Null Count
headlines                   0                0
description                 0                0
content                     0                0
url                         0                0
category                    0                0


In [7]:
display(train_df.duplicated().sum(), test_df.duplicated().sum())

0

0

Number of duplicate for train set rows: 0

Number of duplicate for test set rows: 0

In [8]:
def inspect_dataset(df):
    return {
        "Number of Rows": df.shape[0],
        "Number of Columns": df.shape[1],
        "Total Missing Values": df.isnull().sum().sum(),
        "Duplicate Rows": df.duplicated().sum(),
        "Numerical Columns": df.select_dtypes(include='number').shape[1],
        "Categorical Columns": df.select_dtypes(include='object').shape[1],
    }

train_inspection = inspect_dataset(train_df)
test_inspection = inspect_dataset(test_df)

inspection_results = pd.DataFrame({
    "Metric": train_inspection.keys(),
    "Train Dataset": train_inspection.values(),
    "Test Dataset": test_inspection.values(),
})

print(inspection_results)

                 Metric  Train Dataset  Test Dataset
0        Number of Rows           5520          2000
1     Number of Columns              5             5
2  Total Missing Values              0             0
3        Duplicate Rows              0             0
4     Numerical Columns              0             0
5   Categorical Columns              5             5


In [28]:
train_df.describe().T
#check if there are duplicate 

,count,unique,top,freq
headlines,5520,5512,International Education Day 2024: Know why it ...,2
description,5520,5508,The university has removed the requirements of...,2
content,5520,5513,Grand Slam fever grips tennis fans all over th...,5
url,5520,5514,https://indianexpress.com/article/education/kc...,2
category,5520,5,education,1520


#### 2.2. Data Preprocessing 

2.2.1. Removing stopwords

In [ ]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in stop_words]

for col in columns_to_tokenize:
    train_df[col] = train_df[col].apply(remove_stopwords)
    test_df[col] = test_df[col].apply(remove_stopwords)

print("Train DataFrame (After Removing Stopwords):")
print(train_df.head())

print("\nTest DataFrame (After Removing Stopwords):")
print(test_df.head())

2.2.2. Removing punctuation

In [ ]:
def remove_punctuation(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

train_df['headlines'] = train_df['headlines'].apply(remove_punctuation)
train_df['description'] = train_df['description'].apply(remove_punctuation)
train_df['content'] = train_df['content'].apply(remove_punctuation)

test_df['headlines'] = test_df['headlines'].apply(remove_punctuation)
test_df['content'] = test_df['content'].apply(remove_punctuation)
test_df['content'] = test_df['content'].apply(remove_punctuation)

print("Train DataFrame:")
print(train_df.head())

print("\nTest DataFrame:")
print(test_df.head())

2.2.3. Lowercase

In [12]:
columns_to_lowercase = ['headlines', 'description', 'content']

for col in columns_to_lowercase:
    train_df[col] = train_df[col].str.lower()
    test_df[col] = test_df[col].str.lower()

print("Train DataFrame:")
print(train_df.head())
print("\nTest DataFrame:")

print(test_df.head())

Train DataFrame:
                                           headlines  \
0  rbi revises definition of politicallyexposed p...   
1  ndtv q net profit falls  to rs  crore impacted...   
2  akasa air well capitalised can grow much faste...   
3  indias current account deficit declines sharpl...   
4  states borrowing cost soars to  highest so far...   

                                         description  \
0  the central bank has also asked chairpersons a...   
1  ndtvs consolidated revenue from operations was...   
2  the initial share sale will be open for public...   
3  the current account deficit cad was  per cent ...   
4  the prices shot up reflecting the overall high...   

                                             content  \
0  the reserve bank of india rbi has changed the ...   
1  broadcaster new delhi television ltd on monday...   
2  homegrown server maker netweb technologies ind...   
3  indias current account deficit declined sharpl...   
4  states have been forced to

2.2.4. Tokenisation

In [13]:
columns_to_tokenize = ['headlines', 'description', 'content']

for col in columns_to_tokenize:
    train_df[col] = train_df[col].apply(word_tokenize)
    test_df[col] = test_df[col].apply(word_tokenize)
print(train_df.head())
print(test_df.head())

                                           headlines  \
0  [rbi, revises, definition, of, politicallyexpo...   
1  [ndtv, q, net, profit, falls, to, rs, crore, i...   
2  [akasa, air, well, capitalised, can, grow, muc...   
3  [indias, current, account, deficit, declines, ...   
4  [states, borrowing, cost, soars, to, highest, ...   

                                         description  \
0  [the, central, bank, has, also, asked, chairpe...   
1  [ndtvs, consolidated, revenue, from, operation...   
2  [the, initial, share, sale, will, be, open, fo...   
3  [the, current, account, deficit, cad, was, per...   
4  [the, prices, shot, up, reflecting, the, overa...   

                                             content  \
0  [the, reserve, bank, of, india, rbi, has, chan...   
1  [broadcaster, new, delhi, television, ltd, on,...   
2  [homegrown, server, maker, netweb, technologie...   
3  [indias, current, account, deficit, declined, ...   
4  [states, have, been, forced, to, pay, throu

2.2.5. Lemmatization

In [15]:
lemmatizer = WordNetLemmatizer()

columns_to_process = ['headlines', 'description', 'content']

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

for col in columns_to_process:
  
    train_df[col] = train_df[col].apply(lemmatize_tokens)
    test_df[col] = test_df[col].apply(lemmatize_tokens)

print("Train DataFrame (After Lemmatization):")
print(train_df.head())

print("\nTest DataFrame (After Lemmatization):")
print(test_df.head())
#choosing this lemmatization 

Train DataFrame (After Lemmatization):
                                           headlines  \
0  [rbi, revise, definition, politicallyexposed, ...   
1  [ndtv, q, net, profit, fall, r, crore, impacte...   
2  [akasa, air, well, capitalised, grow, much, fa...   
3  [india, current, account, deficit, decline, sh...   
4  [state, borrowing, cost, soar, highest, far, f...   

                                         description  \
0  [central, bank, also, asked, chairperson, chie...   
1  [ndtvs, consolidated, revenue, operation, r, c...   
2  [initial, share, sale, open, public, subscript...   
3  [current, account, deficit, cad, per, cent, gd...   
4  [price, shot, reflecting, overall, higher, ris...   

                                             content  \
0  [reserve, bank, india, rbi, changed, definitio...   
1  [broadcaster, new, delhi, television, ltd, mon...   
2  [homegrown, server, maker, netweb, technology,...   
3  [india, current, account, deficit, declined, s...   
4  [sta

In [24]:
new_train_df = train_df
new_test_df = test_df

In [25]:
new_train_df.head(5)

,headlines,description,content,url,category
0,"[rbi, revis, definit, politicallyexpos, person...","[central, bank, also, ask, chairperson, chief,...","[reserv, bank, india, rbi, chang, definit, pol...",https://indianexpress.com/article/business/ban...,business
1,"[ndtv, q, net, profit, fall, r, crore, impact,...","[ndtv, consolid, revenu, oper, r, crore, r, cr...","[broadcast, new, delhi, televis, ltd, monday, ...",https://indianexpress.com/article/business/com...,business
2,"[akasa, air, well, capitalis, grow, much, fast...","[initi, share, sale, open, public, subscript, ...","[homegrown, server, maker, netweb, technolog, ...",https://indianexpress.com/article/business/mar...,business
3,"[india, current, account, deficit, declin, sha...","[current, account, deficit, cad, per, cent, gd...","[india, current, account, deficit, declin, sha...",https://indianexpress.com/article/business/eco...,business
4,"[state, borrow, cost, soar, highest, far, fiscal]","[price, shot, reflect, overal, higher, risk, a...","[state, forc, pay, nose, weekli, auction, debt...",https://indianexpress.com/article/business/eco...,business


In [26]:
new_test_df.head(5)

,headlines,description,content,url,category
0,"[nlc, india, win, contract, power, suppli, raj...","[state-own, firm, nlc, india, ltd, (, nlcil, )...","[stateown, firm, nlc, india, ltd, nlcil, monda...",https://indianexpress.com/article/business/com...,business
1,"[sbi, clerk, prelim, exam, date, announc, admi...","[sbi, clerk, prelim, exam, :, sbi, clerk, prel...","[sbi, clerk, prelim, exam, state, bank, indian...",https://indianexpress.com/article/education/sb...,education
2,"[golden, globe, michel, yeoh, ferrel, angela, ...","[barbi, top, nomine, year, ,, follow, close, o...","[michel, yeoh, ferrel, angela, bassett, amanda...",https://indianexpress.com/article/entertainmen...,entertainment
3,"[oneplu, nord, r, part, new, price, cut, here]","[new, deal, make, oneplu, nord, 3, easi, purch...","[review, oneplu, nord, g, prais, balanc, combi...",https://indianexpress.com/article/technology/t...,technology
4,"[adani, famili, partner, use, opaqu, fund, inv...","[cite, review, file, multipl, tax, haven, inte...","[million, dollar, invest, publicli, trade, sto...",https://indianexpress.com/article/business/ada...,business
